# Negamax with Alpha-Beta Pruning and Iterative Deepening

In [1]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [2]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        self.count = 0
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.count = self.count+1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s
    def getWinningValue(self):
        return 1
    def getNumberMovesExplored(self): 
        return self.count

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [3]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

In [44]:
game = TTT()
playGame(game,opponent,20)
print(game.getNumberMovesExplored())

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
558334


# negamaxIDS

In [4]:
def negamaxIDS(game, depthLeft):
    for depth in range(depthLeft):
        score,move = negamax(game,depth)
        if score is None:
            continue
        a = game.getWinningValue()
        if score == a: 
            return score,move
    return score,move
    
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

In [46]:
game = TTT()
playGame(game,opponent,20)
print(game.getNumberMovesExplored())

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
23338


# negamaxIDSab

In [13]:

def negamaxab(game, depthLeft,alpha,beta,checker):
    # If at terminal state or depth limit, return utility value and move None

    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    if checker==True:
        
        for move in game.getMoves():
            # Apply a move to current state
            game.makeMove(move)
            # Use depth-first search to find eventual utility value and back it up.
            #  Negate it because it will come back in context of next player
            value, _ = negamaxab(game, depthLeft-1,alpha,beta,False)
            # Remove the move from current state, to prepare for trying a different move
            game.unmakeMove(move)
            if value is None:
                continue
            if bestValue is None or value > bestValue:
                bestValue, bestMove = -value, move
                # returnState=copy(nextState)
            if bestValue>alpha:
                alpha=bestValue
            if alpha >=beta :
                print ('pruned')
                break;
        
        return bestValue, bestMove

    else:
        for move in game.getMoves():
            # Apply a move to current state
            game.makeMove(move)
            # Use depth-first search to find eventual utility value and back it up.
            #  Negate it because it will come back in context of next player
            value, _ = negamaxab(game, depthLeft-1,alpha,beta,True)
            # Remove the move from current state, to prepare for trying a different move
            game.unmakeMove(move)
            if value is None:
                continue
            if bestValue is None or value < bestValue:
                bestValue, bestMove = -value, move
                # returnState=copy(nextState)
            if bestValue<beta:
                beta=bestValue
            if alpha >=beta :
                print ('pruned')
                break;
        return bestValue, bestMove
    
def negamaxIDSab(game, depthLeft):
    alpha = -float('inf')
    beta = float('inf')
    for depth in range(depthLeft):
        score,move = negamaxab(game,depth,alpha,beta,True)
        if score is None:
            continue
        a = game.getWinningValue()
        if score == a: 
            return score,move
    return score,move
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

In [14]:
game = TTT()
playGame(game,opponent,20)
print(game.getNumberMovesExplored())

 | | 
-----
 | | 
-----
 | | 
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
pruned
pruned
pruned
pruned
pruned
pruned
pruned
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
6053


In [8]:
playGames(opponent, 10)


negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 

negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 

negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
nega